# Pydantic model to create a structured LLM output

## Approach 1

In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

class Country(BaseModel):
    """Information about a country"""
    name: str = Field(description="name of the country")
    language: str = Field(description="language of the country")
    capital: str = Field(description="Capital of the country")

structured_llm = llm.with_structured_output(Country)
structured_llm.invoke("Tell me about France")


Country(name='France', language='French', capital='Paris')

In [2]:
structured_llm

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001F390CC46A0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001F390CD5820>, root_client=<openai.OpenAI object at 0x000001F390A9FF10>, root_async_client=<openai.AsyncOpenAI object at 0x000001F390CC4700>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.Country'>, 'structured_output_format': {'kwargs': {'method': 'json_schema'}, 'schema': {'type': 'function', 'function': {'name': 'Country', 'description': 'Information about a country', 'parameters': {'properties': {'name': {'description': 'name of the country', 'type': 'string'}, 'language': {'description': 'language of the country', 'type': 'string'}, 'capital': {'description': 'Capital of the country', 'type': 'string'}}, 'required': ['name', 'language', 'capital'], 'type': 'object'}}}}}, config={}, conf

# If we don't want validation just need the schema --> go for TypedDict

## Approach 2

In [3]:
from typing_extensions import Annotated, TypedDict
from typing import Optional

class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]
    punchline: Annotated[str, ..., "The punchline of the joke"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]

structured_llm = llm.with_structured_output(Joke)
structured_llm.invoke("Tell me a joke")

{'setup': "Why don't scientists trust atoms?",
 'punchline': 'Because they make up everything!',
 'rating': 8}

## Approach 3

In [4]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline"],
}

structured_llm = llm.with_structured_output(json_schema)
structured_llm.invoke("Tell me a joke about cats")


{'setup': 'Why was the cat sitting on the computer?',
 'punchline': 'It wanted to keep an eye on the mouse!',
 'rating': 8}